In [1]:
import torch
import sys
import numpy as np

In [2]:
print("Python Version:", sys.version)
print("PyTorch Version:", torch.__version__)
print("Cuda Version:", torch.version.cuda)
print("CUDNN Version:", torch.backends.cudnn.version())
print('Numpy Version:', np.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is : {device}")

Python Version: 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]
PyTorch Version: 1.7.1
Cuda Version: 11.0
CUDNN Version: 8005
Numpy Version: 1.19.2
Device is : cuda


In [3]:
# sanity chack preprended 09.06.21
from lftdn.LFT import compact_LFT

class wandbdummy_LFT():
    def __init__(self, stride, win_pad):
        self.stride = stride
        self.window_padding_constrained = win_pad

B, C, H, W = 12, 3, 129, 129
N = 11
torch.manual_seed(2021)
#test_batch = torch.rand((B, C, H, W))
#torch.save(test_batch, 'test_batch.pt')
test_batch = torch.load('test_batch.pt')
test_batch = test_batch.to(device)
test_window = torch.ones((N,N)).to(device)
print('Test batch shape:', test_batch.shape)

test_LFT_config = wandbdummy_LFT(stride=4, win_pad=4)
test_LFT_result = compact_LFT(test_batch, test_window, test_LFT_config)
print('Test result shape:', test_LFT_result.shape)
torch.save(test_LFT_result, 'new_LFT_out.pt')

from asset.utils import getPhaseDiff, getPhaseAdd

test_transform, _ = getPhaseDiff(test_LFT_result, test_LFT_result)
test_LFT_result_transformed = getPhaseAdd(test_LFT_result, test_transform)
print('Test transform shape:', test_transform.shape)

from lftdn.LFT import compact_iLFT

class wandbdummy_m_iLFT():
    def __init__(self, stride, win_pad, H, W):
        self.stride = stride
        self.window_padding_constrained = win_pad
        self.res_y_constrained = H
        self.res_x_constrained = W

test_iLFT_config = wandbdummy_m_iLFT(stride=test_LFT_config.stride, win_pad=test_LFT_config.window_padding_constrained, H=H, W=W)
# move_window introduces considerable noise!
move_window = True
test_reconstruction, _ = compact_iLFT(test_LFT_result_transformed, test_window, test_transform, test_iLFT_config, move_window_according_T=move_window, channel=C)
print('Test reconstruction shape:', test_reconstruction.shape)
torch.save(test_reconstruction, 'new_iLFT_out.pt')

MSE_reconstruction = (test_batch-test_reconstruction)**2
print('MSE:', MSE_reconstruction.mean())
print('MSE max:', MSE_reconstruction.max())

Test batch shape: torch.Size([12, 3, 129, 129])
Test result shape: torch.Size([12, 3888, 19, 19, 2])
Test transform shape: torch.Size([12, 3888, 19, 19, 2])
Test reconstruction shape: torch.Size([12, 3, 129, 129])
MSE: tensor(3.6409e-09, device='cuda:0')
MSE max: tensor(8.1275e-05, device='cuda:0')


In [4]:
from lftdn.LFT import compact_LFT

class wandbdummy_LFT():
    def __init__(self, stride, win_pad):
        self.stride = stride
        self.window_padding_constrained = win_pad

In [5]:
B, C, H, W = 12, 3, 129, 129
N = 11
torch.manual_seed(2021)
test_batch = torch.rand((B, C, H, W))
#test_batch = torch.load('test_batch.pt')
test_batch = test_batch.to(device)
test_window = torch.ones((N,N)).to(device)
print('Test batch shape:', test_batch.shape)

test_LFT_config = wandbdummy_LFT(stride=4, win_pad=4)
test_LFT_result = compact_LFT(test_batch, test_window, test_LFT_config)
print('Test result shape:', test_LFT_result.shape)

Test batch shape: torch.Size([12, 3, 129, 129])
Test result shape: torch.Size([12, 3888, 19, 19, 2])


In [6]:
from asset.utils import getPhaseDiff, getPhaseAdd

test_transform, _ = getPhaseDiff(test_LFT_result, test_LFT_result)
test_LFT_result_transformed = getPhaseAdd(test_LFT_result, test_transform)
print('Test transform shape:', test_transform.shape)

Test transform shape: torch.Size([12, 3888, 19, 19, 2])


In [7]:
from lftdn.LFT import compact_iLFT

class wandbdummy_m_iLFT():
    def __init__(self, stride, win_pad, H, W):
        self.stride = stride
        self.window_padding_constrained = win_pad
        self.res_y_constrained = H
        self.res_x_constrained = W

In [8]:
test_iLFT_config = wandbdummy_m_iLFT(stride=test_LFT_config.stride, win_pad=test_LFT_config.window_padding_constrained, H=H, W=W)
# move_window introduces considerable noise!
move_window = True
test_reconstruction, _ = compact_iLFT(test_LFT_result_transformed, test_window, test_transform, test_iLFT_config, move_window_according_T=move_window, channel=C)
print('Test reconstruction shape:', test_reconstruction.shape)

Test reconstruction shape: torch.Size([12, 3, 129, 129])


In [9]:
MSE_reconstruction = (test_batch-test_reconstruction)**2
print('MSE:', MSE_reconstruction.mean())
print('MSE max:', MSE_reconstruction.max())

MSE: tensor(3.6409e-09, device='cuda:0')
MSE max: tensor(8.1275e-05, device='cuda:0')


In [10]:
from lftdn.custom_fft import custom_fft, custom_ifft

test_batch_fft = custom_fft(test_batch)
print('Test batch just fft shape:', test_batch_fft.shape)
test_reconstruction_fft = custom_ifft(test_batch_fft)
print('Test reconstruction just fft shape:', test_reconstruction_fft.shape)
MSE_reconstruction_fft = (test_batch-test_reconstruction_fft)**2
print('MSE for fft:', MSE_reconstruction_fft.mean())

Test batch just fft shape: torch.Size([12, 3, 129, 129, 2])
Test reconstruction just fft shape: torch.Size([12, 3, 129, 129])
MSE for fft: tensor(1.8512e-14, device='cuda:0')


In [11]:
test_2 = torch.rand((B,C,H,W))
test_2_fft = custom_fft(test_2)

In [12]:
from asset.utils import normalizeFFT

def custom_ifft_old(T, real=True):
    if real:
        return torch.ifft(T, signal_ndim=2, normalized=normalizeFFT)[...,0]
    else:
        return torch.ifft(T, signal_ndim=2, normalized=normalizeFFT)

def custom_fft_old(iT, real=True):
    ###make complex again --- TEMP: use complex signal
    if real:
        iTC = torch.stack([iT, torch.zeros_like(iT,requires_grad=False)], dim=-1)
        return torch.fft(iTC, signal_ndim=2, normalized=normalizeFFT)
    ###convert to modified transformation
    else:
        return torch.fft(iT, signal_ndim=2, normalized=normalizeFFT)

In [13]:
B, C, H, W = 2, 3, 65, 129
test_input_old = torch.rand((B,C,H,W), requires_grad=True)
test_input_new = test_input_old .detach().clone()
test_input_new.requires_grad = True
output_old = custom_fft_old(test_input_old)
ifft_old = custom_ifft_old(output_old)

TypeError: 'module' object is not callable

In [ ]:
#from this point on, old fft cannot be called, as torch.fft becomes a module, not a function
from torch.fft import fftn, ifftn

def custom_ifft_new(T, real=True):
    T = torch.view_as_complex(T)
    if real:
        res = ifftn(T, dim=(-2, -1), norm="forward" if normalizeFFT else "backward")
        return res.real
    else:
        res = ifftn(T, dim=(-2, -1), norm="forward" if normalizeFFT else "backward")
        return torch.view_as_real(res)

def custom_fft_new(iT, real=True):
    ###make complex again --- TEMP: use complex signal
    if real:
        iTC = torch.stack([iT, torch.zeros_like(iT,requires_grad=False)], dim=-1)
        iTC = torch.view_as_complex(iTC)
        res = fftn(iTC, dim=(-2, -1), norm="forward" if normalizeFFT else "backward")
        return torch.view_as_real(res)
    ###convert to modified transformation
    else:
        iT = torch.view_as_complex(iT)
        res = fftn(iT, dim=(-2, -1), norm="forward" if normalizeFFT else "backward")
        return torch.view_as_real(res)

In [ ]:
output_new = custom_fft_new(test_input_new)
ifft_new = custom_ifft_new(output_new)
print(f'Output of old func shape: {output_old.shape}')
print(f'Output of new func shape: {output_new.shape}')
print(f'Are these outputs close?: {((output_old - output_new)**2).mean()}')

print(f'Output of old ifft shape: {ifft_old.shape}')
print(f'Output of new ifft shape: {ifft_new.shape}')
print(f'Are these outputs close?: {((ifft_old - ifft_new)**2).mean()}')

L_old = ((test_input_old - ifft_old)**2).mean()
L_new = ((test_input_new - ifft_new)**2).mean()

print(f'Is the ifft according to old version close to the input? {L_old}')
print(f'Is the ifft according to new version close to the input? {L_new}')

In [ ]:
sum_old = ifft_old.sum()
sum_new = ifft_new.sum()
sum_old.backward()
sum_new.backward()
print(f'Are the gradients of the fft, then ifft sums the same?: {torch.allclose(test_input_old.grad.data, test_input_new.grad.data)}')

OLD FUNC OUT:
Test batch shape: torch.Size([12, 3, 129, 129])
Test result shape: torch.Size([12, 3888, 19, 19, 2])
Test transform shape: torch.Size([12, 3888, 19, 19, 2])
Test reconstruction shape: torch.Size([12, 3, 129, 129])
MSE: tensor(3.6409e-09, device='cuda:0')
MSE max: tensor(8.1275e-05, device='cuda:0')

NEW FUNC OUT:
Test batch shape: torch.Size([12, 3, 129, 129])
Test result shape: torch.Size([12, 3888, 19, 19, 2])
Test transform shape: torch.Size([12, 3888, 19, 19, 2])
Test reconstruction shape: torch.Size([12, 3, 129, 129])
MSE: tensor(3.6409e-09, device='cuda:0')
MSE max: tensor(8.1275e-05, device='cuda:0')

In [14]:
LFT_out_old = torch.load('old_LFT_out.pt')
iLFT_out_old = torch.load('old_iLFT_out.pt')

LFT_out_new = torch.load('new_LFT_out.pt')
iLFT_out_new = torch.load('new_iLFT_out.pt')

print(torch.allclose(LFT_out_old, LFT_out_new) and torch.allclose(iLFT_out_old, iLFT_out_new))

True
